## Feature Engineering

Objective: 
- Check the prediction result and find the factors that might cause prediction error  
- Create features which can resolve error factors

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
DATA_PATH = "./data"

In [4]:
import os

os.listdir(DATA_PATH)

['bert_result',
 'data_eval_labels.csv',
 'data_eval_tfidf.csv',
 'data_eval_word2vec.csv',
 'data_josa_removed.csv',
 'data_preprocessed.csv',
 'data_train_labels.csv',
 'hidden_for_inference.csv',
 'list_josa.txt',
 'list_tags_candidates.txt',
 'sample',
 '몽데이크_Open.csv',
 '중단원_summary.csv']

In [18]:
df_train_label = pd.read_csv(DATA_PATH+'/data_train_labels.csv')
df_train_label.head()

,chapter,bert_80_pred_chapter,tfidf_pred_chapter,word2vec_pred_chapter,text,qtid,name,word2vec_pred_name,qplay_question_id
0,H1S1-08,H1S1-08,H1S1-08,"['H1S1-08', 'H1S1-08', 'H1S1-08']","연립부등식 `{(-x-3 >= -2x+1), (2x+a <= 10) :}`의 해가 ...",H1S1-08-05,해를 갖거나 갖지 않는 연립일차부등식,"['연립이차부등식', '정수인 해의 개수가 주어진 연립부등식', '해를 갖거나 갖지...",3504
1,H1S1-08,H1S1-08,H1S1-08,"['H1S1-08', 'HSU1-06', 'H1S1-08']",부등식 `[x]^2-2[x]-3 < 0`의 해는?,H1S1-08-11,절댓값 또는 가우스 기호가 포함된 이차부등식,"['이차부등식', '삼각부등식 (2)', '절댓값 또는 가우스 기호가 포함된 이차부...",3675
2,H1S1-11,H1S1-11,H1S1-11,"['H1S1-11', 'H1S1-12', 'H1S1-11']","점 `(1, 2)`를 지나며 중심이 직선 `x-y-1=0` 위에 있고 `x`축에 접...",H1S1-11-05,x축 또는 y축에 접하는 원의 방정식,"['x축 또는 y축에 접하는 원의 방정식', '평행이동과 대칭이동의 활용', 'x축...",3779
3,H1S1-07,H1S1-07,H1S1-07,"['H1S1-07', 'H1S1-10', 'H1S1-08']","`x`, `y`가 정수일 때, 방정식 `xy+2x-3y=7`을 만족하는 순서쌍 `(...",H1S1-07-18,정수 및 자연수 조건을 갖는 부정방정식,"['정수 및 자연수 조건을 갖는 부정방정식', '평행한 두 직선 사이의 거리', '...",3649
4,H1S1-12,H1S1-12,H1S1-01,"['H1S1-12', 'H1S1-06', 'H1S2-03']","평행이동 `(x, y)->(x+2, y-7)`에 의해 포물선 `y=x^2`을 평행이...",H1S1-12-03,도형의 평행이동 (2),"['도형의 평행이동 (2)', '식의 최댓값과 최솟값 (2)', '코시-슈바르츠의 ...",3388


In [19]:
token = pd.read_csv(DATA_PATH+'/data_josa_removed.csv')['text']
math_hangul = pd.read_csv(DATA_PATH+'/data_preprocessed.csv')[['math_field', 'hangul_field']]
display(token.head(2))
display(math_hangul.head(2))

0    ['연립부등식', '`', '{', '(', '-', 'x', '-3', '>', ...
1    ['부등식', '`', '[', 'x', ']', '^', '2', '-2', '[...
Name: text, dtype: object

,math_field,hangul_field
0,"['{(-x-3 >= -2x+1), (2x+a <= 10) :}', 'a']","['연립부등식', '해', '존재하도록', '하', '최댓값', '구하']"
1,['[x]^2-2[x]-3 < 0'],"['부등식', '해']"


In [20]:
df_train_label['token'] = token
df_train_label[['math_field', 'hangul_field']] = math_hangul
df_train_label.head()

,chapter,bert_80_pred_chapter,tfidf_pred_chapter,word2vec_pred_chapter,text,qtid,name,word2vec_pred_name,qplay_question_id,token,math_field,hangul_field
0,H1S1-08,H1S1-08,H1S1-08,"['H1S1-08', 'H1S1-08', 'H1S1-08']","연립부등식 `{(-x-3 >= -2x+1), (2x+a <= 10) :}`의 해가 ...",H1S1-08-05,해를 갖거나 갖지 않는 연립일차부등식,"['연립이차부등식', '정수인 해의 개수가 주어진 연립부등식', '해를 갖거나 갖지...",3504,"['연립부등식', '`', '{', '(', '-', 'x', '-3', '>', ...","['{(-x-3 >= -2x+1), (2x+a <= 10) :}', 'a']","['연립부등식', '해', '존재하도록', '하', '최댓값', '구하']"
1,H1S1-08,H1S1-08,H1S1-08,"['H1S1-08', 'HSU1-06', 'H1S1-08']",부등식 `[x]^2-2[x]-3 < 0`의 해는?,H1S1-08-11,절댓값 또는 가우스 기호가 포함된 이차부등식,"['이차부등식', '삼각부등식 (2)', '절댓값 또는 가우스 기호가 포함된 이차부...",3675,"['부등식', '`', '[', 'x', ']', '^', '2', '-2', '[...",['[x]^2-2[x]-3 < 0'],"['부등식', '해']"
2,H1S1-11,H1S1-11,H1S1-11,"['H1S1-11', 'H1S1-12', 'H1S1-11']","점 `(1, 2)`를 지나며 중심이 직선 `x-y-1=0` 위에 있고 `x`축에 접...",H1S1-11-05,x축 또는 y축에 접하는 원의 방정식,"['x축 또는 y축에 접하는 원의 방정식', '평행이동과 대칭이동의 활용', 'x축...",3779,"['점', '`', '(', '1', ',', '2', ')', '`', '지나',...","['(1, 2)', 'x-y-1=0', 'x']","['점', '지나', '중심', '직선', '위', '있', '축', '접하', '..."
3,H1S1-07,H1S1-07,H1S1-07,"['H1S1-07', 'H1S1-10', 'H1S1-08']","`x`, `y`가 정수일 때, 방정식 `xy+2x-3y=7`을 만족하는 순서쌍 `(...",H1S1-07-18,정수 및 자연수 조건을 갖는 부정방정식,"['정수 및 자연수 조건을 갖는 부정방정식', '평행한 두 직선 사이의 거리', '...",3649,"['`', 'x', '`,', '`', 'y', '`', '정수일', '때', ',...","['x', 'y', 'xy+2x-3y=7', '(x, y)']","['정수일', '때', '방정식', '만족하', '순서쌍', '개수']"
4,H1S1-12,H1S1-12,H1S1-01,"['H1S1-12', 'H1S1-06', 'H1S2-03']","평행이동 `(x, y)->(x+2, y-7)`에 의해 포물선 `y=x^2`을 평행이...",H1S1-12-03,도형의 평행이동 (2),"['도형의 평행이동 (2)', '식의 최댓값과 최솟값 (2)', '코시-슈바르츠의 ...",3388,"['평행이동', '`', '(', 'x', ',', 'y', ')', '->', '...","['(x, y)->(x+2, y-7)', 'y=x^2', 'y=x^2+px+q', ...","['평행이동', '의해', '포물선', '평행이동하였더', '포물선', '되었다',..."


In [32]:
wrong_list = []
for i in df.index:
    if df.loc[i, 'chapter'] not in df.loc[i, 'word2vec_pred_chapter']:
        wrong_list.append(i)
        
wrong_df = df.loc[wrong_list]
wrong_df.reset_index(inplace=True)
wrong_df.head(2)

,index,chapter,qtid,text,math_field,hangul_field,tag_field,name,subtitle,qplay_question_id,question_difficulty,question_type_difficulty,grade,purpose,word2vec_pred_chapter,word2vec_pred_name,token
0,7,H1S1-03,H1S1-03-08,다항식 `3x^2+kx+7`이 모든 계수가 정수인 두 일차식의 곱으로 인수분해될 때...,"['3x^2+kx+7', 'k']","['다항식', '모든', '계수', '정수', '두', '일차식', '곱', '인수...",[],조건이 주어진 식의 인수분해,NaN,4002,4,3.0,10,open,"['HSU1-09', 'H1S1-08', 'H1S1-07']","['등비수열의 활용', '이차방정식의 근의 위치', '삼차방정식의 근의 판별']","['다항식', '`', '3', 'x', '^', '2', '+', 'kx', '+..."
1,27,H1S1-05,H1S1-05-12,"`x`에 대한 이차식 `x^2+2x+4`를 복소수의 범위에서 인수분해하였을 때, 인...","['x', 'x^2+2x+4', 'A', 'B', '(A-B)^2']","['대', '이차식', '복소수', '범위', '인수분해하였', '때', '인수',...",[],이차식의 복소수범위 인수분해,NaN,3866,3,2.0,10,open,"['H1S1-01', 'H1S1-11', 'H1S1-11']","['다항식의 덧셈과 뺄셈', '접선의 길이', '원의 방정식 (2)']","['`', 'x', '`', '대', '이차식', '`', 'x', '^', '2'..."


---
wrong_df는 제 word2vec 모델로 중단원을 예측 하였을 때 전혀 맞추지 못한 문제들의 리스트입니다.  
이 worng 데이터를 쭉 살펴보시고 공통점은 무엇인지, 왜 잘못 판단 하였는지 정성적으로 분석해보시면 좋을 것 같습니다.  
모든 데이터를 다 보실 필요는 없고 (너무 힘들겁니다 ㅠ) 쭉 보시며 중요하다 생각되는 부분 위주로 분석해주세요!    

또한 오류를 유발한 factor를 어떻게 제거할 수 있을지도 고민해주시면 좋을 것 같습니다.  
(잘못 판단한 이유로는 토큰화가 잘못되었다던가, 문제에 정보가 부족하다던가, 기호가 잘못 판단되었다던가 하는 문제가 있을 것 같습니다.)  
(참고로 word2vec 모델을 사용할 때에는 name과 token이외에 아무것도 사용하지 않았습니다. 이 두 칼럼 안에 답이 있을 것 같습니다.)
---

### Example)

In [38]:
wrong_df.loc[0, 'text']

'다항식 `3x^2+kx+7`이 모든 계수가 정수인 두 일차식의 곱으로 인수분해될 때, 상수 `k`가 가질 수 있는 값 중 최솟값을 구하시오.]'

In [34]:
wrong_df.loc[0, 'token']

"['다항식', '`', '3', 'x', '^', '2', '+', 'kx', '+', '7', '`', '모든', '계수', '정수', '두', '일차식', '곱', '인수분해될', '때', ',', '상수', '`', 'k', '`', '가질', '수', '있', '값', '중', '최솟값', '구하시오', '.', ']']"

In [36]:
wrong_df.loc[0, 'name']

'조건이 주어진 식의 인수분해'

In [37]:
wrong_df.loc[0, 'word2vec_pred_name']

"['등비수열의 활용', '이차방정식의 근의 위치', '삼차방정식의 근의 판별']"

'인수분해' 라는 단어가 문제에 있음에도 불구하고 엉뚱하게 예측한 것은,  
'인수분해'가 '인수분해될' 로 token화 되어 제대로 인식되지 않은 탓인 것 같습니다.  
'될' 까지 조사로 처리하여 제거할 필요가 있을 것 같습니다.

## word2vec 모델로 예측하지 못한 경우

In [34]:
wrong_list_word2vec = np.zeros(len(df_train_label), dtype=int)

for i in df_train_label.index:
    if df_train_label.loc[i, 'chapter'] not in df_train_label.loc[i, 'word2vec_pred_chapter']:
        wrong_list_word2vec[i] = 1
        
df_train_label['wrong_word2vec'] = wrong_list_word2vec
print("word2vec이 예측하지 못한 문제 개수: {}/{}".format(sum(wrong_list_word2vec), df_train_label.shape[0]))
df_train_label[df_train_label['wrong_word2vec']==1].head()

word2vec이 예측하지 못한 문제 개수: 974/5364


,chapter,bert_80_pred_chapter,tfidf_pred_chapter,word2vec_pred_chapter,text,qtid,name,word2vec_pred_name,qplay_question_id,token,math_field,hangul_field,wrong_word2vec,wrong_tfidf
7,H1S1-03,H1S1-03,H1S1-03,"['HSU1-09', 'H1S1-08', 'H1S1-07']",다항식 `3x^2+kx+7`이 모든 계수가 정수인 두 일차식의 곱으로 인수분해될 때...,H1S1-03-08,조건이 주어진 식의 인수분해,"['등비수열의 활용', '이차방정식의 근의 위치', '삼차방정식의 근의 판별']",4002,"['다항식', '`', '3', 'x', '^', '2', '+', 'kx', '+...","['3x^2+kx+7', 'k']","['다항식', '모든', '계수', '정수', '두', '일차식', '곱', '인수...",1,0
27,H1S1-05,H1S1-05,H1S1-05,"['H1S1-01', 'H1S1-11', 'H1S1-11']","`x`에 대한 이차식 `x^2+2x+4`를 복소수의 범위에서 인수분해하였을 때, 인...",H1S1-05-12,이차식의 복소수범위 인수분해,"['다항식의 덧셈과 뺄셈', '접선의 길이', '원의 방정식 (2)']",3866,"['`', 'x', '`', '대', '이차식', '`', 'x', '^', '2'...","['x', 'x^2+2x+4', 'A', 'B', '(A-B)^2']","['대', '이차식', '복소수', '범위', '인수분해하였', '때', '인수',...",1,0
28,H1S1-05,H1S1-05,H1S1-05,"['HSU1-03', 'H1S1-11', 'H1S1-08']",이차식 `kx^2+4x+k`가 `x`에 대한 완전제곱식이 되기 위한 실수 `k`의 ...,H1S1-05-07,이차식이 완전제곱식이 되는 조건,"['근의 조건이 주어진 지수방정식', '방정식이 원을 나타내기 위한 조건', '이차...",3965,"['이차식', '`', 'kx', '^', '2', '+', '4', 'x', '+...","['kx^2+4x+k', 'x', 'k']","['이차식', '대', '완전제곱식', '되기', '위', '실수', '값', '합']",1,0
30,H1S1-10,H1S1-10,H1S1-10,"['H1S1-11', 'H1S1-11', 'H1S1-12']","점 `(2, 1)`을 지나고 기울기가 `3`인 직선의 방정식을 구하시오?",H1S1-10-01,직선의 방정식 (1),"['x축 또는 y축에 접하는 원의 방정식', '원의 방정식 (1)', '점의 평행이동']",4425,"['점', '`', '(', '2', ',', '1', ')', '`', '지나',...","['(2, 1)', '3']","['점', '지나', '기울기', '직선', '방정식', '구하시오']",1,0
31,H1S1-12,H1S1-12,H1S1-12,"['H1S1-03', 'H1S2-05', 'H1S2-05']","원 `(x-2)^2+(y-3)^2=1`을 점 `M(-1, 1)`에 대하여 대칭이동하...",H1S1-12-09,점에 대한 대칭이동,"['치환을 이용한 인수분해', '유리식과 항등식', '유리식의 사칙연산']",4601,"['원', '`', '(', 'x', '-2', ')', '^', '2', '+',...","['(x-2)^2+(y-3)^2=1', 'M(-1, 1)', '(x-a)^2+(y-...","['원', '점', '대하', '대칭이동하', '옮겨진', '값']",1,0


## tf-idf 모델로 예측하지 못한 경우

In [35]:
wrong_list_tdidf = np.zeros(len(df_train_label), dtype=int)

for i in df_train_label.index:
    if df_train_label.loc[i, 'chapter'] != df_train_label.loc[i, 'tfidf_pred_chapter']:
        wrong_list_tdidf[i] = 1
        
df_train_label['wrong_tfidf'] = wrong_list_tdidf
print("tf-idf가 예측하지 못한 문제 개수: {}/{}".format(sum(wrong_list_tdidf), df_train_label.shape[0]))
df_train_label[df_train_label['wrong_tfidf']==1].head()

tf-idf가 예측하지 못한 문제 개수: 613/5364


,chapter,bert_80_pred_chapter,tfidf_pred_chapter,word2vec_pred_chapter,text,qtid,name,word2vec_pred_name,qplay_question_id,token,math_field,hangul_field,wrong_word2vec,wrong_tfidf
4,H1S1-12,H1S1-12,H1S1-01,"['H1S1-12', 'H1S1-06', 'H1S2-03']","평행이동 `(x, y)->(x+2, y-7)`에 의해 포물선 `y=x^2`을 평행이...",H1S1-12-03,도형의 평행이동 (2),"['도형의 평행이동 (2)', '식의 최댓값과 최솟값 (2)', '코시-슈바르츠의 ...",3388,"['평행이동', '`', '(', 'x', ',', 'y', ')', '->', '...","['(x, y)->(x+2, y-7)', 'y=x^2', 'y=x^2+px+q', ...","['평행이동', '의해', '포물선', '평행이동하였더', '포물선', '되었다',...",0,1
15,H1S1-01,H1S1-01,H1S1-02,"['H1S1-01', 'H1S1-03', 'H1S1-02']",오른쪽 조립제법은 `ax^2+bx+c`를 `(x-p)`로 나눈 몫을 다시 `(x+q...,H1S1-01-09,조립제법,"['조립제법', '치환을 이용한 인수분해', '조립제법과 항등식']",4046,"['오른쪽', '조립제법', '`', 'ax', '^', '2', '+', 'bx'...","['ax^2+bx+c', '(x-p)', '(x+q)', 'ax^2+bx+c', '...","['오른쪽', '조립제법', '나눈', '몫', '다시', '나누', '과정', '...",0,1
17,H1S1-03,H1S1-03,H1S1-01,"['H1S2-05', 'HSU1-01', 'H1S1-03']",`(2060^3-1)/(2061xx2060+1)`의 값은?,H1S1-03-11,인수분해의 활용 (3),"['부분분수로의 변형', '지수의 확장', '인수분해의 활용 (3)']",4383,"['`', '(', '2060', '^', '3', '-1', ')', '/', '...",['(2060^3-1)/(2061xx2060+1)'],['값'],0,1
20,H1S1-10,H1S1-10,H1S1-01,"['H1S1-10', 'H1S1-10', 'H1S1-10']","좌표평면 위의 세 점 `O(0, 0)`, `A(2, 3)`, `B(-3, 5)`를 ...",H1S1-10-15,삼각형의 수심,"['삼각형의 수심', '선분을 수직이등분하는 방정식', '세 점이 한 직선 위에 있...",3594,"['좌표평', '위', '세', '점', '`', 'O', '(', '0', ','...","['O(0, 0)', 'A(2, 3)', 'B(-3, 5)', '△OAB', '(a...","['좌표평', '위', '세', '점', '꼭짓점', '하', '수심', '좌표',...",0,1
23,H1S1-09,H1S1-09,H1S1-01,"['H1S1-09', 'H1S1-10', 'H1S1-10']","좌표평면 위의 두 점 `A(1, 1)`, `B(3, -2)`와 `y`축 위를 움직이...",H1S1-09-05,두 점 사이 거리의 활용,"['같은 거리에 있는 점', '도형의 넓이를 절반으로 나누는 직선의 방정식', '자...",3427,"['좌표평', '위', '두', '점', '`', 'A', '(', '1', ','...","['A(1, 1)', 'B(3, -2)', 'y', 'Q', 'ABQ']","['좌표평', '위', '두', '점', '축', '위', '움직이', '점', '...",0,1


## BERT 모델로 예측하지 못한 경우

In [36]:
wrong_list_bert = np.zeros(len(df_train_label), dtype=int)

for i in df_train_label.index:
    if df_train_label.loc[i, 'chapter'] != df_train_label.loc[i, 'bert_80_pred_chapter']:
        wrong_list_bert[i] = 1
        
df_train_label['wrong_bert'] = wrong_list_bert
print("BERT가 예측하지 못한 문제 개수: {}/{}".format(sum(wrong_list_bert), df_train_label.shape[0]))
df_train_label[df_train_label['wrong_bert']==1].head()

BERT가 예측하지 못한 문제 개수: 13/5364


,chapter,bert_80_pred_chapter,tfidf_pred_chapter,word2vec_pred_chapter,text,qtid,name,word2vec_pred_name,qplay_question_id,token,math_field,hangul_field,wrong_word2vec,wrong_tfidf,wrong_bert
47,H1S1-04,HSU1-06,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']",다음 중 옳은 것은?,H1S1-04-01,복소수의 뜻과 분류,"['명제', '집합의 뜻', '호도법과 육십분법']",4371,"['다음', '중', '옳', '것', '?']",[],"['다음', '중', '옳', '것']",1,1,1
792,HSU1-06,H1S2-01,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']",다음 중 옳은 것은?,HSU1-06-02,삼각함수 값의 대소관계 판별,"['명제', '집합의 뜻', '호도법과 육십분법']",7055,"['다음', '중', '옳', '것', '?']",[],"['다음', '중', '옳', '것']",1,1,1
839,H1S2-06,HSU1-05,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']",다음 중 옳지 않은 것은?,H1S2-06-02,제곱근의 성질,"['명제', '집합의 뜻', '호도법과 육십분법']",16324,"['다음', '중', '옳', '않', '것', '?']",[],"['다음', '중', '옳', '않', '것']",1,1,1
1324,H1S2-01,HSU1-01,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']",다음 중 옳은 것은?,H1S2-01-05,유한집합의 원소의 개수,"['명제', '집합의 뜻', '호도법과 육십분법']",19511,"['다음', '중', '옳', '것', '?']",[],"['다음', '중', '옳', '것']",0,1,1
1460,H1S2-04,HSU1-06,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']",다음 중 옳은 것은?,H1S2-04-08,항등함수와 상수함수,"['명제', '집합의 뜻', '호도법과 육십분법']",24743,"['다음', '중', '옳', '것', '?']",[],"['다음', '중', '옳', '것']",1,1,1


In [62]:
wrong_bert = df_train_label[df_train_label['wrong_bert']==1]
wrong_bert.reset_index(inplace=True)
wrong_bert

,index,chapter,bert_80_pred_chapter,tfidf_pred_chapter,word2vec_pred_chapter,text,qtid,name,word2vec_pred_name,qplay_question_id,token,math_field,hangul_field,wrong_word2vec,wrong_tfidf,wrong_bert,wrong_sum
0,47,H1S1-04,HSU1-06,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']",다음 중 옳은 것은?,H1S1-04-01,복소수의 뜻과 분류,"['명제', '집합의 뜻', '호도법과 육십분법']",4371,"['다음', '중', '옳', '것', '?']",[],"['다음', '중', '옳', '것']",1,1,1,3
1,792,HSU1-06,H1S2-01,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']",다음 중 옳은 것은?,HSU1-06-02,삼각함수 값의 대소관계 판별,"['명제', '집합의 뜻', '호도법과 육십분법']",7055,"['다음', '중', '옳', '것', '?']",[],"['다음', '중', '옳', '것']",1,1,1,3
2,839,H1S2-06,HSU1-05,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']",다음 중 옳지 않은 것은?,H1S2-06-02,제곱근의 성질,"['명제', '집합의 뜻', '호도법과 육십분법']",16324,"['다음', '중', '옳', '않', '것', '?']",[],"['다음', '중', '옳', '않', '것']",1,1,1,3
3,1324,H1S2-01,HSU1-01,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']",다음 중 옳은 것은?,H1S2-01-05,유한집합의 원소의 개수,"['명제', '집합의 뜻', '호도법과 육십분법']",19511,"['다음', '중', '옳', '것', '?']",[],"['다음', '중', '옳', '것']",0,1,1,2
4,1460,H1S2-04,HSU1-06,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']",다음 중 옳은 것은?,H1S2-04-08,항등함수와 상수함수,"['명제', '집합의 뜻', '호도법과 육십분법']",24743,"['다음', '중', '옳', '것', '?']",[],"['다음', '중', '옳', '것']",1,1,1,3
5,1629,H1S1-03,HSU1-05,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']",다음 중 옳지 않은 것은?,H1S1-03-01,인수분해의 기본,"['명제', '집합의 뜻', '호도법과 육십분법']",4950,"['다음', '중', '옳', '않', '것', '?']",[],"['다음', '중', '옳', '않', '것']",1,1,1,3
6,1915,H1S1-03,HSU1-01,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']",다음 중 옳은 것은?,H1S1-03-01,인수분해의 기본,"['명제', '집합의 뜻', '호도법과 육십분법']",4471,"['다음', '중', '옳', '것', '?']",[],"['다음', '중', '옳', '것']",1,1,1,3
7,2209,H1S2-04,H1S2-05,H1S2-04,"['H1S2-05', 'H1S2-05', 'H1S2-04']","함수 `f(x)=(x-1)/x`에 대하여 `f^1=f`, `f^(n+1)=f@f^n` (`n`은 자연수)이고, `f^2018(x)`의 역함수를 `g(x)`라고 할 때, `g(2018)`의 값은?",H1S2-04-16,역함수의 정의,"['유리함수의 합성', '유리함수의 합성함수와 역함수', '합성함수와 역함수']",25074,"['함수', '`', 'f', '(', 'x', ')', '=', '(', 'x', '-1', ')', '/', 'x', '`', '대하', '`', 'f', '^', '1', '=', 'f', '`,', '`', 'f', '^', '(', 'n', '+', '1', ')', '=', 'f', '@', 'f', '^', 'n', '`', '(', '`', 'n', '`', '자연수', ')', ',', '`', 'f', '^', '2018', '(', 'x', ')', '`', '역함수', '`', 'g', '(', 'x', ')', '`', '할', '때', ',', '`', 'g', '(', '2018', ')', '`', '값', '?']","['f(x)=(x-1)/x', 'f^1=f', 'f^(n+1)=f@f^n', 'n', 'f^2018(x)', 'g(x)', 'g(2018)']","['함수', '대하', '자연수', '역함수', '할', '때', '값']",0,0,1,1
8,2229,H1S2-01,HSU1-01,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']",다음 중 옳은 것은?,H1S2-01-05,유한집합의 원소의 개수,"['명제', '집합의 뜻', '호도법과 육십분법']",21213,"['다음', '중', '옳', '것', '?']",[],"['다음', '중', '옳', '것']",0,1,1,2
9,2304,H1S2-06,H1S2-04,H1S2-06,"['H1S2-06', 'H1S2-04', 'H1S2-05']","정의역이 `{x| x > 1}`인 두 함수<br/>`f(x)=(x+2)/(x-1)`, `g(x)=sqrt(2x-2)+1`<br/>에 대하여 `(f@(g@f)^-1@f)(2)`의 값은?",H1S2-06-14,무리함수의 합성함수와 역함수,"['무리함수의 합성함수와 역함수', '역함수의 성질', '유리함수의 합성']",24967,"['정의역', '`', '{', 'x', '|', 'x', '>', '1', '}', '`', '두', '함수', '`', 'f', '(', 'x', ')', '=', '(', 'x', '+', '2', ')', '/', '(', 'x', '-1', ')', '`,', '`', 'g', '(', 'x', ')', '=', 'sqrt', '(', '2', 'x', '-2', ')', '+', '1', '`', '대하', '`', '(', 'f', '@', '(', 'g', '@', 'f', ')', '^', '-1', '@', 'f', ')', '(', '2', ')', '`', '값', '?']","['{x| x > 1}', 'f(x)=(x+2)/(x-1)', 'g(x)=sqrt(2x-2)+1', '(f@(g@f)^-1@f)(2)']","['정의역', '두', '함수', '대하', '값']",0,0,1,1


In [65]:
# index 7,9,10 을 제외한 나머지 문제는 '다음 중 옳은 것은/다음 중 옳지 않은 것은'
wrong_bert.loc[7,:]

index                    2209                                                                                                                                                                                                                                                                                                                                                                        
chapter                  H1S2-04                                                                                                                                                                                                                                                                                                                                                                     
bert_80_pred_chapter     H1S2-05                                                                                                                                                                                            

## wrong_sum

In [39]:
# wrong_sum: 3가지 모델에서 잘못 예측한 경우의 count
df_train_label['wrong_sum'] = df_train_label['wrong_word2vec'] + df_train_label['wrong_tfidf'] + df_train_label['wrong_bert']
df_train_label['wrong_sum'].value_counts()

0    3974
1    1187
2     196
3       7
Name: wrong_sum, dtype: int64

---

# 3가지 모델 모두에서 예측하지 못한 경우

+ 모두 `'다음 중 옳은 것은?'/ '다음 중 옳지 않은 것은'` 에 대한 문제.
    + chapter(실제) : H1S1-03, H1S1-04, H1S2-04, H1S2-06, HSU1-06	
    + BERT: 다양한 값을 예측했음. But how?
    + tfidf_pred_chapter: H1S1-01	
    + word2vec_pred_chapter: 'H1S2-03', 'H1S2-01', 'HSU1-05'
    
+ BERT가 그나마 가장 좋은 예측... 별다른 방법이 없을 것 같다. 

In [46]:
print("3가지 모델 모두에서 예측하지 못한 경우:", len(df_train_label[df_train_label['wrong_sum'] == 3]))
df_train_label[df_train_label['wrong_sum'] == 3][['qplay_question_id', 'chapter', 'text', 'bert_80_pred_chapter', 'tfidf_pred_chapter', 'word2vec_pred_chapter']]

3가지 모델 모두에서 예측하지 못한 경우: 7


,qplay_question_id,chapter,text,bert_80_pred_chapter,tfidf_pred_chapter,word2vec_pred_chapter
47,4371,H1S1-04,다음 중 옳은 것은?,HSU1-06,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']"
792,7055,HSU1-06,다음 중 옳은 것은?,H1S2-01,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']"
839,16324,H1S2-06,다음 중 옳지 않은 것은?,HSU1-05,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']"
1460,24743,H1S2-04,다음 중 옳은 것은?,HSU1-06,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']"
1629,4950,H1S1-03,다음 중 옳지 않은 것은?,HSU1-05,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']"
1915,4471,H1S1-03,다음 중 옳은 것은?,HSU1-01,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']"
4605,7059,HSU1-06,다음 중 옳은 것은?,H1S2-01,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']"


모두 `'다음 중 옳은 것은?'/ '다음 중 옳지 않은 것은'` 에 대한 문제를 맞춘 경우?

In [56]:
search = '다음 중 옳은 것은?'

# 조건를 충족하는 데이터를 필터링
subset_df = df_train_label[df_train_label['text'] == search]

print("search 개수:", subset_df.shape[0])
print(subset_df['chapter'].value_counts())
subset_df[['qplay_question_id', 'chapter', 'text', 'wrong_bert', 'wrong_tfidf', 'wrong_word2vec', 'wrong_sum']]

search 개수: 10
HSU1-01    3
H1S2-01    2
HSU1-06    2
H1S2-04    1
H1S1-03    1
H1S1-04    1
Name: chapter, dtype: int64


,qplay_question_id,chapter,text,wrong_bert,wrong_tfidf,wrong_word2vec,wrong_sum
35,5459,HSU1-01,다음 중 옳은 것은?,0,1,1,2
47,4371,H1S1-04,다음 중 옳은 것은?,1,1,1,3
792,7055,HSU1-06,다음 중 옳은 것은?,1,1,1,3
1324,19511,H1S2-01,다음 중 옳은 것은?,1,1,0,2
1460,24743,H1S2-04,다음 중 옳은 것은?,1,1,1,3
1561,5473,HSU1-01,다음 중 옳은 것은?,0,1,1,2
1609,5460,HSU1-01,다음 중 옳은 것은?,0,1,1,2
1915,4471,H1S1-03,다음 중 옳은 것은?,1,1,1,3
2229,21213,H1S2-01,다음 중 옳은 것은?,1,1,0,2
4605,7059,HSU1-06,다음 중 옳은 것은?,1,1,1,3


In [57]:
search = '다음 중 옳지 않은 것은?'

# 조건를 충족하는 데이터를 필터링
subset_df = df_train_label[df_train_label['text'] == search]

print("search 개수:", subset_df.shape[0])
print(subset_df['chapter'].value_counts())
subset_df[['qplay_question_id', 'chapter', 'text', 'wrong_bert', 'wrong_tfidf', 'wrong_word2vec', 'wrong_sum']]

search 개수: 5
HSU1-05    2
H1S2-06    1
H1S1-03    1
HSU1-06    1
Name: chapter, dtype: int64


,qplay_question_id,chapter,text,wrong_bert,wrong_tfidf,wrong_word2vec,wrong_sum
193,8044,HSU1-05,다음 중 옳지 않은 것은?,0,1,0,1
839,16324,H1S2-06,다음 중 옳지 않은 것은?,1,1,1,3
1629,4950,H1S1-03,다음 중 옳지 않은 것은?,1,1,1,3
2598,7946,HSU1-05,다음 중 옳지 않은 것은?,1,1,0,2
2696,7024,HSU1-06,다음 중 옳지 않은 것은?,0,1,1,2


---

# 2가지 모델에서 예측하지 못한 경우

+ 모두 `'다음 중 옳은 것은?'/ '다음 중 옳지 않은 것은'` 에 대한 문제.
    + chapter(실제) : H1S1-03, H1S1-04, H1S2-04, H1S2-06, HSU1-06	
    + BERT: 다양한 값을 예측했음. But how?
    + tfidf_pred_chapter: H1S1-01	
    + word2vec_pred_chapter: 'H1S2-03', 'H1S2-01', 'HSU1-05'
    
+ BERT가 그나마 가장 좋은 예측... 별다른 방법이 없을 것 같다. 

In [59]:
pd.set_option('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', -1)

print("2가지 모델 모두에서 예측하지 못한 경우:", len(df_train_label[df_train_label['wrong_sum'] == 2]))
df_train_label[df_train_label['wrong_sum'] == 2][['qplay_question_id', 'chapter', 'text', 'bert_80_pred_chapter', 'tfidf_pred_chapter', 'word2vec_pred_chapter']]

2가지 모델 모두에서 예측하지 못한 경우: 196


C:\Users\sdn11\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,qplay_question_id,chapter,text,bert_80_pred_chapter,tfidf_pred_chapter,word2vec_pred_chapter
35,5459,HSU1-01,다음 중 옳은 것은?,HSU1-01,H1S1-01,"['H1S2-03', 'H1S2-01', 'HSU1-05']"
117,4271,H1S1-12,"직선 `2x-y=0`을 직선 `x+y-3=0`에 대하여 대칭이동한 직선의 방정식이 `x+ay+b=0`일 때, 상수 `a`, `b`에 대하여 `a^2+b^2`의 값은?",H1S1-12,H1S1-10,"['H1S1-10', 'H1S1-10', 'H1S1-06']"
143,7295,HSU1-03,"두 집합 `A={x|5^(2x)-20xx5^x-125 <= 0}`, `B={x|(1/9)^x-8(1/3)^x-9 < 0}`에 대하여 집합 `AnnB`에 속하는 모든 정수 원소의 합을 구하시오.",HSU1-03,H1S2-02,"['H1S2-02', 'H1S2-03', 'H1S2-02']"
184,4087,H1S1-08,"이차방정식 `x^2+(a-1)x+(a-1)=0`이 중근을 가질 때, 모든 실수 `a`의 값의 합은?",H1S1-08,H1S1-01,"['H1S1-02', 'H1S1-05', 'H1S1-02']"
189,5434,H1S1-07,"영희와 준희가 사탕 주머니에서 각각 사탕을 `1`개 이상 꺼냈다. 꺼낸 사탕 개수를 기준으로 철수에게는 영희와 준희가 꺼낸 사탕 개수의 합의 `4`배 만큼을 주고, 나미에게는 영희가 꺼낸 사탕 개수의 제곱과 준희가 꺼낸 사탕 개수의 제곱의 합만큼을 주고, 진수에게는 영희가 꺼낸 사탕 개수와 준희가 꺼낸 사탕 개수의 곱의 `2`배 만큼을 주었다고 할 때 철수, 나미, 진수에게 준 사탕 개수의 합이 `12`이다. 이때 영희가 꺼낸 사탕 개수는?",H1S1-07,H1S1-01,"['HSTA-03', 'HSTA-03', 'HSTA-02']"
218,4727,H1S1-10,"네 점 `O(0, 0)`, `A(4, 1)`, `B(6, 8)`, `C(2, 7)`을 꼭짓점으로 하는 사각형 `OABC`의 넓이는?",H1S1-10,H1S1-09,"['H1S1-09', 'H1S1-09', 'H1S1-09']"
229,4214,H1S1-07,"다음 그림과 같이 서로 외접하는 작은 두 구가 반지름의 길이가 `6`인 큰 구에 내접하고 있다. 작은 두 구의 겉넓이의 합이 `112pi`일 때, 작은 두 구의 부피의 합은 `kpi`이다. 실수 `k`의 값을 구하시오.",H1S1-07,H1S1-01,"['HSU1-05', 'H1S1-05', 'H1S1-08']"
265,8553,HSU1-01,"`a > 0`, `b > 0`일 때, `root4 (ab) divide root4 (a^3b^2) xx sqrt(asqrtb)`을 간단히 하면?",HSU1-01,H1S1-01,"['H1S2-05', 'H1S1-03', 'H1S1-01']"
282,7056,HSU1-09,"`A`, `B` 두 사람은 각각 같은 농도의 식염수 `9`L씩 가지고 있다. `A`는 식염수 `6`L를 버리고 물 `6`L를 넣는 시행을 반복하고, `B`는 식염수 `8`L를 버리고 물 `8`L를 넣는 시행을 반복한다. `A`, `B`가 시행을 각각 `p`번, `q`번 반복하여 얻은 식염수의 농도가 서로 같을 때, `p:q`를 가장 간단한 자연수의 비로 나타내면?",HSU1-09,H1S1-01,"['H1S1-07', 'HSTA-04', 'HSTA-04']"
298,7215,HSU1-06,"예각삼각형 `ABC`의 세 내각의 크기를 `A`, `B`, `C`라고 할 때, 다음 중 항상 옳은 것이 아닌 것을 고르면?",HSU1-06,H1S1-01,"['H1S2-02', 'H1S2-02', 'H1S2-02']"


In [66]:
from modules.preprocess_for_kobert import preprocess

df_train_label = preprocess(df_train_label)

In [70]:
df_train_label.loc[591, :]

chapter                  HSU1-02                                                                                                                                                                                                                                 
bert_80_pred_chapter     HSU1-02                                                                                                                                                                                                                                 
tfidf_pred_chapter       H1S1-01                                                                                                                                                                                                                                 
word2vec_pred_chapter    ['H1S2-05', 'H1S2-05', 'H1S2-04']                                                                                                                                                                        

In [74]:
df_train_label.loc[635, :]

chapter                  HSU1-02                                                                                                                                                                                                                                                                                                                  
bert_80_pred_chapter     HSU1-02                                                                                                                                                                                                                                                                                                                  
tfidf_pred_chapter       H1S1-01                                                                                                                                                                                                                                                                                                  

In [72]:
df = pd.read_csv("data/몽데이크_Open.csv")
df = preprocess(df)

In [75]:
df.loc[635, :]

qplay_question_id           8735                                                                                        
text                        `[log_(2)1]+[log_(2)2]+[log_(2)3]+…+[log_(2)256]`의 값을 구하여라. (단, `[x]`는 `x`를 넘지 않는 최대의 정수이다.)
qtid                        HSU1-02-10                                                                                  
name                        로그의 정수 부분과 소수 부분                                                                            
subtitle                    NaN                                                                                         
question_difficulty         4                                                                                           
question_type_difficulty    2                                                                                           
grade                       10                                                                                          
purpose                     open

In [ ]:
log 관련된 문제에서 

BERT 두 모델에서 토큰화된거 출력하는 방법 찾을 예정
word2vec과 tfidf 쪽 분석 계속 진행할까요